- gcp-automl: https://github.com/googleapis/python-automl/blob/356eb12708f8219644160f59775e4aae18b66f24/google/cloud/automl_v1beta1/services/prediction_service/client.py#L83

# group pdf

## steps

In [ ]:
from glob import glob
import os

### mkdir

In [36]:
subject_list = ['國文', '英文', '數學', '自然', '社會', '英語']
press_list = ['康軒', '南一', '翰林']
qa_list = ['題', '答']
# for subject in subject_list:
#     for press in press_list:
#         dirname = os.path.join('.', 'datasets', 'pdfs', '108', subject, press)
#         os.makedirs(dirname, exist_ok=True)

### move file

In [38]:
for subject in subject_list:
    for press in press_list:
        for qa in qa_list:
            dirname = os.path.join('.', 'datasets', 'pdfs', '108', subject, press, qa)
            os.makedirs(dirname, exist_ok=True)

            pdf_path_list = glob(os.path.join('.', 'datasets', 'pdfs', '108', '*', '*', '*%s*%s*%s*.pdf')%(press, subject, qa))
            for pdf_path in pdf_path_list[:]:
                old_path = pdf_path
                old_dirname = os.path.dirname(old_path)
                old_filename = os.path.basename(old_path)

                new_dirname = dirname
                new_filename = old_filename.replace(' ', '')
                new_path = os.path.join(new_dirname, old_filename)

                os.rename(old_path, new_path)


## refactor

# pdf to paper

## step by step

In [ ]:
from glob import glob
import os
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

from pdf2image import convert_from_path, convert_from_bytes


In [ ]:
pdf_path_list = sorted(glob(os.path.join('.', 'datasets', 'pdf', '*')))
pdf_path = pdf_path_list[-1]
pdf_path

In [ ]:
images = convert_from_path(pdf_path)


In [ ]:
'''
0 1-1
1 1-2
2 2-1
3 2-2
4 3-1
5 3-2
'''


In [ ]:
pdf_name = os.path.basename(pdf_path).replace('.pdf', '')
pdf_name

In [ ]:
from IPython.display import Image 

for i, img in enumerate(images[:4]):
    a = i // 2 + 1
    b = i % 2 + 1
    page = '%s_%s'%(a, b)
    filename = '%s-%s.png'%(pdf_name, page)
    img_path = os.path.join('.', 'datasets', 'img', filename)
    img.save(img_path, 'PNG')

## refactor

In [ ]:
from glob import glob
import os
from pdf2image import convert_from_path

class MyPdf2Paper:
    def __init__(self, 
                 pdf_dirname='./datasets/pdf', 
                 img_dirname='./datasets/paper'):
        
        self.pdf_dirname = pdf_dirname
        self.img_dirname = img_dirname
    
    def get_pdf_path_list(self, is_question=True):
        if is_question:
            pdf_path_list = sorted(glob(os.path.join(self.pdf_dirname, '*-(題).*')))
        else:
            pdf_path_list = sorted(glob(os.path.join(self.pdf_dirname, '*-(答).*')))
        return pdf_path_list
        
    def pdf2images(self, pdf_path):
        images = convert_from_path(pdf_path)
        return images
    
    def save_images(self, images, pdf_path):
        pdf_name = os.path.basename(pdf_path).replace('.pdf', '')
        pdf2excludePageList = self.get_pdf2excludePageList()
        exclude_page_list = pdf2excludePageList[pdf_name]
        
        for i, img in enumerate(images):
            page = i+1
            if page in exclude_page_list: continue
                
            a = i // 2 + 1
            b = i % 2 + 1
            page_name = '%s_%s'%(a, b)
            filename = '%s-%s.png'%(pdf_name, page_name)
            img_path = os.path.join(self.img_dirname, filename)
            img.save(img_path, 'PNG')
            
    def get_pdf2excludePageList(self):
        pdf2excludePageList = {
            '108上[翰林]國中試卷-(三)數學-B卷-(中上)-(題)': [],
            '108上[翰林]國中試卷-(三)數學-A卷-(優等)-(題)': [29, 30],            
            '108上[康軒]國中試卷-(三)數學-E卷-(最難)-(題)': [],
            '108上[康軒]國中試卷-(三)數學-B卷-(中上)-(題)': [],
            '108上[康軒]國中試卷-(三)數學-A卷-(優等)-(題)': [],
            '108上[南一]國中試卷-(三)數學-B卷-(中上)-(題)': [33, 34],
            '108上[南一]國中試卷-(三)數學-A卷-(優等)-(題)': []
        }
        return pdf2excludePageList
            


In [ ]:
myPdf2Paper = MyPdf2Paper()
pdf_path_list = myPdf2Paper.get_pdf_path_list(is_question=True)
pdf_path_list


In [ ]:
# pdf_path = pdf_path_list[1]
# pdf_path
# images = myPdf2Paper.pdf2images(pdf_path, )
# myPdf2Paper.save_images(images, pdf_path, exclude_page_list=[33, 34])

In [ ]:
for pdf_path in pdf_path_list[-2:]:
    print(pdf_path)
    images = myPdf2Paper.pdf2images(pdf_path)
    myPdf2Paper.save_images(images, pdf_path)
    

# paper to prediction

## step by step

In [ ]:
import sys

from google.cloud import automl_v1beta1
# from google.cloud.automl_v1beta1.proto import service_pb2

import os
path = '/home/ryh/dummy/genuine-octagon-302302-86a342e2b291.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path



In [ ]:
# 'content' is base-64-encoded image data.
def get_prediction(content, project_id, model_id):
    prediction_client = automl_v1beta1.PredictionServiceClient()

    name = 'projects/{}/locations/us-central1/models/{}'.format(project_id, model_id)
    payload = {'image': {'image_bytes': content }}
    params = {}
    request = prediction_client.predict(name=name, payload=payload, params=params)
    return request  # waits till request is returned


In [ ]:
import base64

filename = '108上[南一]國中試卷-(三)數學-B卷-(中上)-(題)-1_1.png'
img_path = os.path.join('.', 'datasets', 'img', filename) 
    
with open(img_path, 'rb') as ff:
    content = ff.read()
    
len(content), type(content)

In [ ]:
import json
import proto

In [ ]:
paper_path_list = sorted(glob('./datasets/paper/*'))
paper_path = paper_path_list[0]
paper_path

In [ ]:
project_id = 'genuine-octagon-302302'
model_id = 'IOD1970652491439669248'
for paper_path in paper_path_list[:]:
    print(paper_path)
    prediction = get_prediction(content, project_id, model_id)
    json_string = proto.Message.to_json(prediction)
    json_dict = json.loads(json_string)
    
    json_path = paper_path.replace('paper', 'json').replace('.png', '.json')
    with open(json_path, 'w') as fp:
        json.dump(json_dict, fp)
    

## refactor

In [ ]:
from google.cloud import automl_v1beta1
from glob import glob
import sys
import os
import json
import proto
path = '/home/ryh/dummy/genuine-octagon-302302-86a342e2b291.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path

class MyPaper2Prediction:
    def __init__(self, ):
        pass
    
    def get_prediction(self, 
                       content, 
                       project_id='genuine-octagon-302302', 
                       model_id='IOD1970652491439669248'):
        
        prediction_client = automl_v1beta1.PredictionServiceClient()
        name = 'projects/{}/locations/us-central1/models/{}'.format(project_id, model_id)
        payload = {'image': {'image_bytes': content }}
        params = {}
        request = prediction_client.predict(name=name, payload=payload, params=params)
        return request  # waits till request is returned

    def get_paper_path_list(self):
        paper_path_list = sorted(glob('./datasets/paper/*'))
        return paper_path_list
    
    def save_prediction(self, paper_path):
        with open(paper_path, 'rb') as f:
            content = f.read()
    
        prediction = self.get_prediction(content)
        json_string = proto.Message.to_json(prediction)
        json_dict = json.loads(json_string)

        json_path = paper_path.replace('paper', 'prediction').replace('.png', '.json')
        with open(json_path, 'w') as fp:
            json.dump(json_dict, fp)        


In [ ]:
myPaper2Prediction = MyPaper2Prediction()
paper_path_list = myPaper2Prediction.get_paper_path_list()
len(paper_path_list)

In [ ]:
for i, paper_path in enumerate(paper_path_list[100:]):
    print(i+1, len(paper_path_list), end='\r')
    myPaper2Prediction.save_prediction(paper_path)

In [ ]:
236*3/60

# paper to question

## step by step

In [ ]:
import json

In [ ]:
paper_path = './datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題)-15_1.png'
pred_path = paper_path.replace('paper', 'prediction').replace('.png', '.json')


In [ ]:
with open(pred_path, 'r') as fp:
    pred = json.load(fp)
    
annotation_list = pred['payload']

annotation_id = 8
annotation = annotation_list[annotation_index]
imageObjectDetection = annotation['imageObjectDetection']
boundingBox = imageObjectDetection['boundingBox']
normalizedVertices = boundingBox['normalizedVertices']
normalizedVertices

In [ ]:
# Improting Image class from PIL module 
from PIL import Image 
  
# Opens a image in RGB mode 
paper = Image.open(paper_path) 

# Size of the image in pixels (size of orginal image) 
# (This is not mandatory) 
width, height = paper.size 
width, height

In [ ]:
left_top = normalizedVertices[0]
right_bottom = normalizedVertices[1]

left, top = left_top['x'], left_top['y']
right, bottom = right_bottom['x'], right_bottom['y']


In [ ]:
# Setting the points for cropped image 
left = left*width
top = top*height
right = right*width
bottom = bottom*height
  
# Cropped image of above dimension 
# (It will not change orginal image) 
paper_crop = paper.crop((left, top, right, bottom)) 
paper_crop

In [ ]:
type(paper_crop)

In [ ]:
question_path = paper_path.replace('paper', 'question').replace('.png', '') + '-%s'%(annotation_id) + '.png'
question_path


In [ ]:
paper_crop.save(question_path)

In [ ]:

# import io

# img_byte_arr = io.BytesIO()
# paper_crop.save(img_byte_arr, format='PNG')
# img_byte_arr = img_byte_arr.getvalue()
# type(img_byte_arr)

In [ ]:
from google.cloud import vision
import io

In [ ]:
def detect_text(paper_crop):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()
    
    img_byte_arr = io.BytesIO()
    paper_crop.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    image = vision.Image(content=img_byte_arr)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    return texts[0].description

In [ ]:
# text = detect_text(paper_crop)
# text

In [ ]:
import re

def get_q_id(text):
    # txt = ') 2. 23. 設二'
    p = '([1-9]|[1-9][0-9])\.'
    x = re.search(p, text)
    if x==None: 
        return str(x)
    else:
        return x.group(1)

In [ ]:
import pandas as pd

In [ ]:
columns = ['paper_path', 'annotation_index', 'question_path', 'question_text', 'normalizedVertices', ]
log = pd.DataFrame(columns=columns)
# log['paper_path'] = [paper_path]
# log['annotation_index'] = [annotation_index]
# log['question_path'] = [question_path]
# log['question_text'] = [text]
# log['normalizedVertices'] = [normalizedVertices]

log

In [ ]:
log = pd.read_csv('./datasets/log.csv')

In [ ]:
log['q_id'] = log['question_text'].apply(lambda x: get_q_id(x))
log

In [ ]:
paper_path_list = log.paper_path.unique()
paper_path_list

## refactor

In [ ]:
import json
import io
import os
from glob import glob
import pandas as pd
from PIL import Image 
from google.cloud import vision

class MyPaper2Question:
    def __init__(self):
        self.log_path = './datasets/log.pickle'
        self.log = self.get_log()
        self.client = vision.ImageAnnotatorClient()
    
    def process_paper(self, paper_path):
        paper = self.get_paper(paper_path)
        annotation_list = self.get_annotation_list(paper_path)
        for annotation_id in range(len(annotation_list[:])):
            question_path = paper_path.replace('paper', 'question').replace('.png', '') + '-%s'%(annotation_id) + '.png'
            
            annotation = annotation_list[annotation_id]
            normalizedVertices = self.get_normalizedVertices(annotation)
            
            question = self.process_question(normalizedVertices, paper, question_path)
            
            self.save_log(paper_path, question_path, annotation_id, normalizedVertices, question)
    
    def get_paper_path_list(self):
        paper_path_list = sorted(glob('./datasets/paper/*'))
        return paper_path_list
    
    def get_log(self):
        if os.path.exists(self.log_path):
            log = pd.read_csv(self.log_path)
        else:
            columns = ['paper_path', 'annotation_id', 'question_path', 'question_text', 'vertex1', 'vertex2']
            log = pd.DataFrame(columns=columns)
        return log
    
    def get_paper(self, paper_path):
        paper = Image.open(paper_path)
        return paper
        
    def get_annotation_list(self, paper_path):
        pred_path = paper_path.replace('paper', 'prediction').replace('.png', '.json')
        with open(pred_path, 'r') as fp:
            pred = json.load(fp)
        annotation_list = pred['payload']
        return annotation_list
    
    def get_normalizedVertices(self, annotation):
        imageObjectDetection = annotation['imageObjectDetection']
        boundingBox = imageObjectDetection['boundingBox']
        normalizedVertices = boundingBox['normalizedVertices']
        return normalizedVertices
        
    def process_question(self, normalizedVertices, paper, question_path):
        width, height = paper.size
        left = normalizedVertices[0]['x']*width
        top = normalizedVertices[0]['y']*height
        right = normalizedVertices[1]['x']*width
        bottom = normalizedVertices[1]['y']*height
        question = paper.crop((left, top, right, bottom)) 
        question.save(question_path)
        return question
        
    def detect_text(self, paper_crop):
        img_byte_arr = io.BytesIO()
        paper_crop.save(img_byte_arr, format='PNG')
        img_byte_arr = img_byte_arr.getvalue()

        image = vision.Image(content=img_byte_arr)
        response = self.client.text_detection(image=image)
        texts = response.text_annotations
        return texts[0].description
    
    def save_log(self, paper_path, question_path, annotation_id, normalizedVertices, question):
        all_q = set(self.log.question_path)
        if question_path in all_q: 
            print(question_path, 'already exists')
            return
        
        text = self.detect_text(question)
        
        row = pd.DataFrame()
        row['paper_path'] = [paper_path]
        row['annotation_id'] = [annotation_id]
        row['question_path'] = [question_path]
        row['question_text'] = [text]
#         row['normalizedVertices'] = [normalizedVertices]
        row['vertex1'] = [(normalizedVertices[0]['x'], normalizedVertices[0]['y'])]
        row['vertex2'] = [(normalizedVertices[1]['x'], normalizedVertices[1]['y'])]
        self.log = self.log.append(row)
        self.log.to_pickle(self.log_path)
        

In [ ]:
myPaper2Question = MyPaper2Question()

In [ ]:
paper_path_list = myPaper2Question.get_paper_path_list()

paper_name = '108上[南一]國中試卷-(三)數學-B卷-(中上)-(題)-15_1.png'
paper_path = './datasets/paper/%s'%(paper_name)
paper_path_list.index(paper_path)

paper_path_list = paper_path_list[42:42+4]
paper_path_list

In [ ]:
for paper_path in paper_path_list:
    myPaper2Question.process_paper(paper_path)

# naming question

## step by step

In [2]:
import json
import io
import os
import re
from glob import glob
import pandas as pd



In [3]:
def get_lv2_id(text):
    p = '([1-9]|[1-9][1-9][0-9])\.'
    x = re.search(p, text)
    if x==None: 
        return 'x'
    else:
        return int(x.group(1))

text = ') 123 . 設二'
get_lv2_id(text)

'x'

In [4]:
log = pd.read_pickle('./datasets/log.pickle')
log['onePaperID'] = 'x'
log['lv1_id'] = 'x'
log['lv2_id'] = 'x'

log['lv2_id'] = log['question_text'].apply(lambda x: get_lv2_id(x))
log = log[log.lv2_id != 'x']
log

,paper_path,annotation_id,question_path,question_text,vertex1,vertex2,onePaperID,lv1_id,lv2_id
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,0,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,3. 4- (x--V3 )²=3• U\n,"(0.5126525, 0.71513087)","(0.845159, 0.7625979)",x,x,3
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,1,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,"1. 多項式A=22x-9x-1,則下列哪一個選項為\nA的因式?\n(A) 22x–1\n...","(0.10500417, 0.10221725)","(0.48196438, 0.1975474)",x,x,1
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,2,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,3. 下列哪一個方程式的解與其他不同?\n(A) ( x- 3 )=16\n(B) (x-1...,"(0.10475833, 0.3686036)","(0.40541813, 0.45320252)",x,x,3
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,3,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,"8. 解(3x-7)?+8(3x-7)+16=0,則此方程式\n的兩根為何?\n(A) 4或...","(0.56326663, 0.09013641)","(0.93681, 0.18804573)",x,x,8
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,4,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,"2. 已知a為整數,且多項式2x+ax-5可因式分\n解成兩個整數係數的一次式乘積。若下列三...","(0.10491303, 0.20305745)","(0.48541027, 0.36601004)",x,x,2
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,5,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,"6. 對於方程式( 3x+1)(x-2)=(2x-1)(x-2)\n的敘述,下列何者正確?\...","(0.10401165, 0.7200054)","(0.49578378, 0.83423877)",x,x,6
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,6,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,"2.若a、b為(一6x+5) (x-4)=(3x+2) (x-4)的\n解,且a<b,則 a=\n","(0.5045662, 0.58743376)","(0.9259302, 0.6401858)",x,x,2
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,7,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,"0 5. 若a、b為一元二次方程式x(4x+9)=0的兩\n根,且a>b,則b-a= ?\n...","(0.10111798, 0.566962)","(0.47665703, 0.71541554)",x,x,5
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,8,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,") 4. 設二次方程式 ax +4x+2=0的兩根相等,則\n6-a的值為何?\n(A) O...","(0.098322764, 0.45542043)","(0.47580793, 0.56549466)",x,x,4
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,10,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,"7. 若一4與1是一元二次方程式tax+b=0的\n兩根,則\n2a+b\n= ?\n(A)...","(0.10576627, 0.8347987)","(0.48134336, 0.9461565)",x,x,7


### one_paper

In [5]:
def sort_one_paper(row):
    vertex1 = row['vertex1']
    x, y = vertex1
    if x < 0.4: return (0, y)
    else: return (1, y)

def inplace_update_df(df1, df2, key_col='a', value_col='b'):
    dic = dict(zip(df2[key_col], df2[value_col]))
    df1[value_col] = df1.apply(lambda row: dic[row[key_col]] if row[key_col] in dic else row[value_col], axis=1)

def get_one_paper(log, paper_path):
    one_paper = log[log.paper_path==paper_path].copy()
    one_paper['temp'] = one_paper.apply(lambda row: sort_one_paper(row), axis=1)
    one_paper = one_paper.sort_values(by='temp').reset_index(drop=True)
    one_paper['onePaperID'] = one_paper.index
    return one_paper


# one_paper = get_one_paper(log, paper_path)
# inplace_update_df(log, one_paper, key_col='question_path', value_col='onePaperID')


In [6]:
paper_path_list = log.paper_path.unique()
paper_path_list

array(['./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題)-15_1.png',
       './datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題)-15_2.png',
       './datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題)-16_1.png',
       './datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題)-16_2.png'],
      dtype=object)

In [7]:
for paper_path in paper_path_list:
    if log[log.paper_path==paper_path].onePaperID.values[0] != 'x': 
        print(paper_path, '###', 'already done')
        continue
    one_paper = get_one_paper(log, paper_path)
    inplace_update_df(log, one_paper, key_col='question_path', value_col='onePaperID')


In [ ]:
log

### two_paper

In [8]:
import re
def is_paper_1(text):
    p = '-([1-9]|[1-9][0-9])_1.png'
    x = re.search(p, text)
    if x!=None: return True
    else: return False
    
# text = './datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題)-99_2.png'
# is_paper_1(text)
paper_1_path_list = [x for x in log.paper_path.unique() if is_paper_1(x)]
paper_1_path = paper_1_path_list[0]


In [9]:
def get_two_paper(paper_1_path, log):
    paper_2_path = paper_1_path.replace('_1.png', '_2.png')
    two_paper = log[log.paper_path.isin([paper_1_path, paper_2_path])]
    two_paper = two_paper.sort_values(by=['paper_path', 'onePaperID']).reset_index(drop=True)
    
    lv1_id = 1
    two_paper.loc[0, 'lv1_id'] = lv1_id
    for i in range(1, len(two_paper)):
        if int(two_paper.loc[i, 'lv2_id']) <= int(two_paper.loc[i-1, 'lv2_id']):
            lv1_id += 1
        two_paper.loc[i, 'lv1_id'] = lv1_id   

    return two_paper




In [13]:
for paper_1_path in paper_1_path_list:
    two_paper = get_two_paper(paper_1_path, log)
    inplace_update_df(log, two_paper, key_col='question_path', value_col='lv1_id')

In [14]:
log

,paper_path,annotation_id,question_path,question_text,vertex1,vertex2,onePaperID,lv1_id,lv2_id
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,0,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,3. 4- (x--V3 )²=3• U\n,"(0.5126525, 0.71513087)","(0.845159, 0.7625979)",11,2,3
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,1,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,"1. 多項式A=22x-9x-1,則下列哪一個選項為\nA的因式?\n(A) 22x–1\n...","(0.10500417, 0.10221725)","(0.48196438, 0.1975474)",0,1,1
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,2,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,3. 下列哪一個方程式的解與其他不同?\n(A) ( x- 3 )=16\n(B) (x-1...,"(0.10475833, 0.3686036)","(0.40541813, 0.45320252)",2,1,3
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,3,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,"8. 解(3x-7)?+8(3x-7)+16=0,則此方程式\n的兩根為何?\n(A) 4或...","(0.56326663, 0.09013641)","(0.93681, 0.18804573)",7,1,8
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,4,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,"2. 已知a為整數,且多項式2x+ax-5可因式分\n解成兩個整數係數的一次式乘積。若下列三...","(0.10491303, 0.20305745)","(0.48541027, 0.36601004)",1,1,2
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,5,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,"6. 對於方程式( 3x+1)(x-2)=(2x-1)(x-2)\n的敘述,下列何者正確?\...","(0.10401165, 0.7200054)","(0.49578378, 0.83423877)",5,1,6
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,6,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,"2.若a、b為(一6x+5) (x-4)=(3x+2) (x-4)的\n解,且a<b,則 a=\n","(0.5045662, 0.58743376)","(0.9259302, 0.6401858)",10,2,2
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,7,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,"0 5. 若a、b為一元二次方程式x(4x+9)=0的兩\n根,且a>b,則b-a= ?\n...","(0.10111798, 0.566962)","(0.47665703, 0.71541554)",4,1,5
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,8,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,") 4. 設二次方程式 ax +4x+2=0的兩根相等,則\n6-a的值為何?\n(A) O...","(0.098322764, 0.45542043)","(0.47580793, 0.56549466)",3,1,4
0,./datasets/paper/108上[南一]國中試卷-(三)數學-B卷-(中上)-(題...,10,./datasets/question/108上[南一]國中試卷-(三)數學-B卷-(中上)...,"7. 若一4與1是一元二次方程式tax+b=0的\n兩根,則\n2a+b\n= ?\n(A)...","(0.10576627, 0.8347987)","(0.48134336, 0.9461565)",6,1,7


## refactor